## Import Libraries

In [250]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import *

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.optimizers import Adam

## Import Data

In [251]:
# Reading Processed Train Set Data File
raw_d1 = pd.read_csv('Data/Processed_Trainset.csv')

# Feature Headers List
headers = list(raw_d1.columns)

# Reading Test Data Set
raw_d2 = pd.read_csv('Data/Processed_Testset.csv')

id_col = list(raw_d2.Client_ID)
raw_d2.drop(['Client_ID'], axis=1, inplace=True)

In [252]:
# Convert Data into ndArrays
train_data = np.array(raw_d1)
test_data = np.array(raw_d2)

# Total Training Data
X_tot = train_data[:, :-1]
Y_tot = train_data[:, -1]

# Splitting Data into a Training Set and a Cross Validation Set
X_train, X_cross, Y_train, Y_cross = train_test_split(X_tot, Y_tot, test_size = 0.2, shuffle = True)
X_train.shape, Y_train.shape

((19200, 20), (19200,))

## SMOTE

In [253]:
# categorical_features = list(range(0,18))

# smote = SMOTENC(categorical_features, sampling_strategy='auto', k_neighbors=10, n_jobs=1)
# X_train,Y_train =  smote.fit_resample(X_train, Y_train)

# pd.DataFrame(np.concatenate((X_train,np.reshape(Y_train, (Y_train.shape[0], 1))), axis=1)
#              ,columns=headers).to_csv('Data/SMOTED.csv', index=None)
# X_train.shape, Y_train.shape

## Normalizing

In [254]:
sc = RobustScaler()
sc.fit(X_tot)

X_tot = sc.fit_transform(X_tot)
X_train = sc.transform(X_train)
X_cross = sc.transform(X_cross)
test_data = sc.transform(test_data)

pd.DataFrame(X_tot, columns=headers[:-1])

,Balance_Limit_V1,g_F,g_M,es_Graduate,es_High School,es_Other,ms_Other,ms_Single,age_31-45,age_46-65,age_Less than 30,age_More than 65,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,Paid_AMT,Due_AMT
0,0.750,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.292543,-0.142447
1,0.750,1.0,-1.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,5.225175,0.693545
2,-0.375,1.0,-1.0,0.0,1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,2.0,-2.0,-2.0,-0.547214,-0.326830
3,-0.250,1.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,-0.168610,0.383002
4,0.750,1.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.947225,3.131199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0.750,1.0,-1.0,0.0,1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.731680,1.995211
23996,0.000,1.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.116380,-0.331620
23997,-0.375,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,-1.0,0.0,0.0,0.443874,-0.189588
23998,-0.250,1.0,-1.0,0.0,1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,3.958804,0.442754


## NN Model

In [260]:
model = Sequential()
model.add(Dense(30, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [261]:
history = model.fit(X_train, Y_train, epochs=200, batch_size=X_train.shape[0], validation_data=(X_cross, Y_cross))

Train on 19200 samples, validate on 4800 samples
Epoch 1/200
19200/19200 [==============================] - 1s 43us/sample - loss: 1.0422 - acc: 0.2142 - val_loss: 0.7550 - val_acc: 0.4602
Epoch 2/200
19200/19200 [==============================] - 0s 1us/sample - loss: 0.7596 - acc: 0.4592 - val_loss: 0.6183 - val_acc: 0.7658
Epoch 3/200
19200/19200 [==============================] - 0s 1us/sample - loss: 0.6236 - acc: 0.7625 - val_loss: 0.5698 - val_acc: 0.7831
Epoch 4/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.5762 - acc: 0.7766 - val_loss: 0.5479 - val_acc: 0.7831
Epoch 5/200
19200/19200 [==============================] - 0s 1us/sample - loss: 0.5556 - acc: 0.7766 - val_loss: 0.5391 - val_acc: 0.7831
Epoch 6/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.5476 - acc: 0.7766 - val_loss: 0.5373 - val_acc: 0.7831
Epoch 7/200
19200/19200 [==============================] - 0s 1us/sample - loss: 0.5461 - acc: 0.7766 - val_loss: 0.

Epoch 60/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4347 - acc: 0.8193 - val_loss: 0.4395 - val_acc: 0.8173
Epoch 61/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4344 - acc: 0.8190 - val_loss: 0.4393 - val_acc: 0.8175
Epoch 62/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4341 - acc: 0.8192 - val_loss: 0.4390 - val_acc: 0.8171
Epoch 63/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4337 - acc: 0.8193 - val_loss: 0.4388 - val_acc: 0.8173
Epoch 64/200
19200/19200 [==============================] - 0s 1us/sample - loss: 0.4334 - acc: 0.8195 - val_loss: 0.4387 - val_acc: 0.8175
Epoch 65/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4331 - acc: 0.8195 - val_loss: 0.4387 - val_acc: 0.8177
Epoch 66/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4328 - acc: 0.8193 - val_loss: 0.4387 - val_acc: 0.8183
Epoch 67/200
19200/1

Epoch 119/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4209 - acc: 0.8219 - val_loss: 0.4358 - val_acc: 0.8177
Epoch 120/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4207 - acc: 0.8218 - val_loss: 0.4359 - val_acc: 0.8173
Epoch 121/200
19200/19200 [==============================] - 0s 1us/sample - loss: 0.4205 - acc: 0.8218 - val_loss: 0.4359 - val_acc: 0.8173
Epoch 122/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4204 - acc: 0.8219 - val_loss: 0.4360 - val_acc: 0.8171
Epoch 123/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4202 - acc: 0.8221 - val_loss: 0.4361 - val_acc: 0.8169
Epoch 124/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4200 - acc: 0.8224 - val_loss: 0.4362 - val_acc: 0.8169
Epoch 125/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4198 - acc: 0.8226 - val_loss: 0.4362 - val_acc: 0.8169
Epoch 126/200

Epoch 178/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4121 - acc: 0.8232 - val_loss: 0.4400 - val_acc: 0.8156
Epoch 179/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4120 - acc: 0.8233 - val_loss: 0.4402 - val_acc: 0.8160
Epoch 180/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4118 - acc: 0.8234 - val_loss: 0.4402 - val_acc: 0.8158
Epoch 181/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4118 - acc: 0.8234 - val_loss: 0.4409 - val_acc: 0.8160
Epoch 182/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4119 - acc: 0.8235 - val_loss: 0.4406 - val_acc: 0.8146
Epoch 183/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4122 - acc: 0.8235 - val_loss: 0.4427 - val_acc: 0.8150
Epoch 184/200
19200/19200 [==============================] - 0s 2us/sample - loss: 0.4132 - acc: 0.8240 - val_loss: 0.4416 - val_acc: 0.8148
Epoch 185/200

## Prediction and Evaluation

In [262]:
def predict_y(X_set):
    threshold = 0.5
    return np.where(model.predict(X_set)>= threshold, 1, 0)

predict_train = predict_y(X_train)
predict_cross = predict_y(X_cross)
predict_tot = predict_y(X_tot)
predict_test = predict_y(test_data)

In [263]:
# Computing Confusion Matrix on Cross Validation Set
confusion = confusion_matrix(Y_cross,predict_cross)

# Computing Accuracy Values
accuracy_train = accuracy_score(Y_train, predict_train)
accuracy_cross = accuracy_score(Y_cross, predict_cross)
accuracy_tot = accuracy_score(Y_tot, predict_tot)

# Computing AUC Values
auc_train = roc_auc_score(Y_train, predict_train)
auc_cross = roc_auc_score(Y_cross, predict_cross)
auc_tot = roc_auc_score(Y_tot, predict_tot)                

# Printing Accuracy Metrics
print("Train Accuracy            : {:.3f}%       AUC: {:.4f}".format(accuracy_train * 100, auc_train))
print("Cross Validation Accuracy : {:.3f}%       AUC: {:.4f} ".format(accuracy_cross * 100, auc_cross))
print("Total Set Accuracy        : {:.3f}%       AUC: {:.4f}".format(accuracy_tot * 100, auc_tot))
print("Confusion Matrix for Cross Validation Set :\n %s" % str(confusion))
print("Number of Ones Predicted for Test Data : %d" % np.sum(predict_test))

print(classification_report(Y_cross, predict_cross))

Train Accuracy            : 82.396%       AUC: 0.6678
Cross Validation Accuracy : 81.396%       AUC: 0.6534 
Total Set Accuracy        : 82.196%       AUC: 0.6650
Confusion Matrix for Cross Validation Set :
 [[3522  237]
 [ 656  385]]
Number of Ones Predicted for Test Data : 702
              precision    recall  f1-score   support

         0.0       0.84      0.94      0.89      3759
         1.0       0.62      0.37      0.46      1041

    accuracy                           0.81      4800
   macro avg       0.73      0.65      0.68      4800
weighted avg       0.79      0.81      0.80      4800



## Upload

In [264]:
# Uploading Predictions to a CSV
df = pd.DataFrame(predict_test, dtype=int)
df.index = id_col
df.to_csv('Data/SubmissionNN.csv', header=['NEXT_MONTH_DEFAULT'], index_label='Client_ID')